# Calculating Value at Risk and Conditional VaR

We will be calculating VaR using three methods
 - Historical Method
 - Parametric Method
 - Monte Carlo Method

## Gathering Historical Data

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
from scipy.stats import norm, t
import matplotlib.pyplot as plt

In [26]:
# Import data
def getData(stocks, start, end):
    stockData = yf.download(stocks, start=start, end=end)
    stockData = stockData['Close']
    log_returns = np.log(stockData / stockData.shift(1))  # Calculate log returns
    meanReturns = log_returns.mean()
    covMatrix = log_returns.cov()
    return log_returns, meanReturns, covMatrix

In [27]:
# Portfolio Performance
def portfolioPerformance(weights, meanReturns, covMatrix, Time):
    log_returns = np.sum(meanReturns*weights)*Time
    std = np.sqrt( np.dot(weights.T, np.dot(covMatrix, weights)) ) * np.sqrt(Time)
    return log_returns, std

In [28]:
stockList = ['CBA', 'BHP', 'TLS', 'NAB', 'WBC', 'STO']
stocks = [stock+'.AX' for stock in stockList]
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=800)

log_returns, meanlog_returns, covMatrix = getData(stocks, start=startDate, end=endDate)
log_returns = log_returns.dropna()

#randomly weighting the portolio
weights = np.random.random(len(log_returns.columns))
weights /= np.sum(weights)

log_returns['portfolio'] = log_returns.dot(weights)

[*********************100%%**********************]  6 of 6 completed


## Historical Method for VaR

Creating the function for  Historical Var

In [29]:
def historicalVar(log_returns, alpha = 5):
  if isinstance(log_returns, pd.Series):
    return np.percentile(log_returns, alpha)

  elif isinstance(log_returns, pd.DataFrame):
    return log_returns.aggregate(historicalVar, alpha =5)

  else:
    raise TypeError("Log Returns should be DF or Series")

Checking Historical VaR for our portfolio

In [30]:
print(historicalVar(log_returns, alpha =5))

Ticker
BHP.AX      -0.028647
CBA.AX      -0.020155
NAB.AX      -0.020974
STO.AX      -0.027132
TLS.AX      -0.013122
WBC.AX      -0.020322
portfolio   -0.015547
dtype: float64


In [31]:
print(historicalVar(log_returns['portfolio'], alpha =5))

-0.015546905843632688


Creating function for Historical CVar

In [32]:
def historicalCVar(log_returns, alpha = 5):
  if isinstance(log_returns, pd.Series):
    belowVaR = log_returns <= historicalVar(log_returns, alpha=alpha)
    return log_returns[belowVaR].mean()

  elif isinstance(log_returns, pd.DataFrame):
    return log_returns.aggregate(historicalCVar, alpha =5)

  else:
    raise TypeError("Log Returns should be DF or Series")

Checking Historical CVaR for our portfolio

In [33]:
print(historicalCVar(log_returns, alpha =5))

Ticker
BHP.AX      -0.043195
CBA.AX      -0.027775
NAB.AX      -0.029707
STO.AX      -0.042358
TLS.AX      -0.021695
WBC.AX      -0.029366
portfolio   -0.021678
dtype: float64


In [34]:
print(historicalCVar(log_returns['portfolio'], alpha =5))

-0.02167776664760457


Overall information

In [35]:
pRet, pStd = portfolioPerformance(weights, meanlog_returns, covMatrix, 365)

In [36]:
InitialInvestment = 10000
print('Expected Portfolio Return:      ', round(InitialInvestment*pRet,2))
print('Value at Risk 95th CI    :      ', round(InitialInvestment*historicalVar(log_returns['portfolio'], alpha =5),2))
print('Conditional VaR 95th CI  :      ', round(InitialInvestment*historicalCVar(log_returns['portfolio'], alpha =5),2))

Expected Portfolio Return:       718.43
Value at Risk 95th CI    :       -155.47
Conditional VaR 95th CI  :       -216.78


If we invest \$10,000 in the portfolio, based on our analysis using log returns, we can anticipate a return of approximately \$718.43. With 95% confidence, we estimate that our maximum potential loss would be around \$155.47. However, in the remaining 5% of scenarios, we could experience a larger loss of approximately \$216.78. These estimates provide insights into the potential risks and rewards associated with our investment strategy, taking into account the properties of log returns.

## Parametric Method for VaR

## Monte Carlo Method for VaR